In [1]:
import sqlite3
import csv

In [2]:
# Getting data from csv file
lines_with_value = []
# is list of pair( list[ words in line/article] , list[its annotated value])
with open('/home/balu/cl2/project/datasets/telugu/test.csv', 'r') as total_file:
    data = csv.reader(total_file)
    next(data)
    for line in data:
        for k in reversed(line):
            if(k==''):
                del line[-1:]
            else:
                break
        i = len(line)
        k = (line[i-2],line[i-1])
        del line[-2:]
        temp = []
        for cell in line:
            cell = cell.split()
            for token in cell:
                temp.append(token)
        lines_with_value.append((temp,k))

print(lines_with_value)

[(['వ్యవసాయానికి', '‘ప్రత్యేకం’', 'ముఖ్యమంత్రి', 'కేసీఆర్\u200c', 'వెల్లడి', 'తెలంగాణ', 'రాష్ట్రంలో', 'ఈ', 'నెల', '25', '26', 'తేదీల్లో', 'మండల', 'రైతు', 'సమన్వయ', 'సమితిల', 'ప్రాంతీయ', 'సదస్సులు', 'నిర్వహించాలని', 'రాష్ట్ర', 'ముఖ్యమంత్రి', 'కేసీఆర్\u200c', 'నిర్ణయించారు.', 'వ్యవసాయాన్ని', 'లాభసాటిగా', 'చేసేందుకు', 'ప్రభుత్వం', 'తీసుకుంటున్న', 'చర్యలను', 'రైతులకు', 'చేర్చే', 'విషయంలో', 'రైతు', 'సమన్వయ', 'సమితిలు', 'కీలక', 'పాత్ర', 'పోషించాలని', 'ఆయన', 'సూచించారు.', 'ఈనెల', '25న', 'హైదరాబాద్\u200cలోని', 'వ్యవసాయ', 'విశ్వ', 'విద్యాలయంలో', '26న', 'కరీంనగర్', 'అంబేడ్కర్', 'స్టేడియంలో', 'ప్రాంతీయ', 'సదస్సులు', 'నిర్వహిస్తామని', 'వెల్లడించారు.', 'ప్రగతి', 'భవన్', 'లో', 'ఆదివారం', 'వ్యవసాయ', 'శాఖపై', 'ముఖ్యమంత్రి', 'కేసీఆర్', 'సమీక్ష', 'నిర్వహించారు.', 'ఈ', 'బడ్జెట్\u200cలో', 'వ్యవసాయానికి', 'ప్రత్యేక', 'బడ్జెట్', 'ప్రవేశపెట్టాలని', 'నిర్ణయించినందున', 'ఇందుకు', 'సంబంధించిన', 'ముసాయిదా', 'తయారు', 'చేయాలని', 'అధికారులను', 'ఆదేశించారు.', 'పంట', 'పెట్టుబడి', 'మద్దతు', 'పథకానికి', 'సంబంధించిన', 'ఆ

In [3]:

#Creating vectors for all words in the given training dataset
import itertools
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from tensorflow import keras
import tensorflow as tf
# Drawing the embeddings
import matplotlib
import matplotlib.pyplot as plt

# Deep learning: 
from keras.models import Model
from keras import Input
from keras.layers import Dense

from scipy import sparse

#function for text processing
def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''')->list:
    processed_text = []
    text = ['వ్యవసాయానికి ‘ప్రత్యేకం’ ముఖ్helయమంత్రి కేసీఆర్\u200c వెల్లడి', 'తెలంగాణ రాష్ట్రంలో ఈ నెల 25', '26 తేదీల్లో మండల రైతు సమన్వయ సమితిల ప్రాంతీయ సదస్సులు నిర్వహించాలని రాష్ట్ర ముఖ్యమంత్రి కేసీఆర్\u200c నిర్ణయించారు. వ్యవసాయాన్ని లాభసాటిగా చేసేందుకు ప్రభుత్వం తీసుకుంటున్న చర్యలను రైతులకు చేర్చే విషయంలో రైతు సమన్వయ సమితిలు కీలక పాత్ర పోషించాలని ఆయన సూచించారు. ఈనెల 25న హైదరాబాద్\u200cలోని వ్యవసాయ విశ్వ విద్యాలయంలో', '26న కరీంనగర్ అంబేడ్కర్ స్టేడియంలో ప్రాంతీయ సదస్సులు నిర్వహిస్తామని వెల్లడించారు. ప్రగతి భవన్ లో ఆదివారం వ్యవసాయ శాఖపై ముఖ్యమంత్రి కేసీఆర్ సమీక్ష నిర్వహించారు. ఈ బడ్జెట్\u200cలో వ్యవసాయానికి ప్రత్యేక బడ్జెట్ ప్రవేశపెట్టాలని నిర్ణయించినందున ఇందుకు సంబంధించిన ముసాయిదా తయారు చేయాలని అధికారులను ఆదేశించారు. పంట పెట్టుబడి మద్దతు పథకానికి సంబంధించిన ఆర్థిక సాయాన్ని చెక్కు రూపంలో రెండు విడతలుగా అందించనున్నట్లు  వెల్లడించారు. వ్యవసాయంలో యాంత్రీకరణను ప్రోత్సహించేందుకు సబ్సిడీపై నాట్లు వేసే యంత్రాలు అందివ్వాలని యోచిస్తున్నట్లు ఈ సందర్భంగా ముఖ్యమంత్రి తెలిపారు. అన్ని జిల్లాలకు ప్రాతినిధ్యం కల్పిస్తూ', 'వ్యవసాయ రంగాభివృద్ధి కోసం కృషి చేస్తున్న నిపుణులు', 'శాస్త్రవేత్తలతో కూడిన రాష్ట్ర స్థాయి రైతు సమన్వయ సమితిని త్వరలోనే నియమిస్తామని ప్రకటించారు. ఈ సమీక్షా సమావేశంలో ఆర్థిక శాఖ మంత్రి ఈటల రాజేందర్\u200c', 'ప్రభుత్వ ముఖ్య సలహాదారు రాజీవ్ శర్మ', 'ప్రభుత్వ ప్రధాన కార్యదర్శి ఎస్.కె.జోషి', 'ఎంపీలు గుత్తా సుఖేందర్ రెడ్డి', 'బాల్క సుమన్', 'ప్రభుత్వ విప్ పల్లా రాజేశ్వర్ రెడ్డి', 'మిషన్ భగీరథ వైస్ చైర్మన్ వేముల ప్రశాంత్ రెడ్డి', 'కార్పొరేషన్ల చైర్మన్లు బాలమల్లు', 'శేరి సుభాష్ రెడ్డి', 'ఎమ్మెల్సీ కర్నె ప్రభాకర్', 'ప్రభుత్వ ముఖ్య కార్యదర్శులు ఎస్.నర్సింగ్ రావు', 'రామకృష్ణ రావు', 'పార్థసారథి', 'వ్యవసాయ శాఖ కమిషనర్ జగన్మోహన్ రావు', 'వ్యవసాయ యూనివర్సిటీ విసి ప్రవీణ్ రావు', 'ఆర్థిక సలహాదారు జిఆర్ రెడ్డి', 'సిఎంఓ కార్యదర్శులు స్మితా సభర్వాల్', 'రాజశేఖర్ రెడ్డి', 'భూపాల్ రెడ్డి తదితరులు పాల్గొన్నారు.']

    for x in text:
        if x in punctuations:
            text = text.replace(x,"")
        
        # Removing words that have numbers in them
        x = re.sub(r'\w*\d\w*', '', x)
        x=x.replace('\u200c','')
        x = re.sub(r'[a-z]+', '', x)
        # Removing digits
        x = re.sub(r'[0-9]+', '', x)

        # Cleaning the whitespaces
        x = re.sub(r'\s+', ' ', x).strip()
        # print(type(x))
        # Converting all our x to a list 
        x = x.split(' ')
        # Droping empty strings
        x = [y for y in x if x!='']

        for y in x:
            processed_text.append(y)
    return processed_text


def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict 




# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists = []
all_text = []
texts = []
# for x in lines_with_value:
#     for y in x[0]:
#         texts.append(y)
# print (texts)
for text in lines_with_value:

    # Cleaning the text
    text = text_preprocessing(text[0])
    # print(text)
    # Appending to the all text list
    for word in text:
        all_text.append(word)

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text): 
                word_lists.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [text[(i - w - 1)]])

unique_word_dict = create_unique_word_dict(all_text)

# Defining the number of features (unique words)
n_words = len(unique_word_dict)
# print(all_text)
# print(n_words)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into a sparse format because the vast majority of the data are 0s
XX = sparse.csr_matrix(X)
YY = sparse.csr_matrix(Y)
print(type(X))
#Defining the embed size
embed_size = 2

#Defining Neural Network
inp = Input(shape=(XX.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=YY.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# X=tf.convert_to_tensor(X)
# Y=tf.convert_to_tensor(Y)
X = np.array(X)
Y=np.array(Y)
# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    batch_size=256,
    epochs=10
    )

# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]

# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })
# matplotlib.rcParams['font.sans-serif'] = ['Source Han Sans TW',
#                                    'sans-serif',
#                                    "Arial Unicode MS"  # fc-list :lang=hi family
#                                    ]
# # Ploting the embeddings
# plt.figure(figsize=(10, 10))
# plt.set_anno
# for word in list(unique_word_dict.keys()):
#     coord = embedding_dict.get(word)
#     plt.scatter(coord[0], coord[1])
#     plt.annotate(word, (coord[0], coord[1]))

2022-10-15 21:23:38.293195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 21:23:38.566546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 21:23:38.566578: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-15 21:23:38.604378: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 21:23:39.445111: W tensorflow/stream_executor/platform/de

<class 'list'>
Epoch 1/10
19/19 [==============================] - 0s 2ms/step - loss: 5.0485
Epoch 2/10
19/19 [==============================] - 0s 2ms/step - loss: 5.0431
Epoch 3/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0380
Epoch 4/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0330
Epoch 5/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0278
Epoch 6/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0224
Epoch 7/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0168
Epoch 8/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0108
Epoch 9/10
19/19 [==============================] - 0s 1ms/step - loss: 5.0044
Epoch 10/10
19/19 [==============================] - 0s 1ms/step - loss: 4.9975


In [12]:
# TRAININIG BERNOULLI AND TESTING IT
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import time
import seaborn as sns
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
    xticklabels = categories, yticklabels = categories)
    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

# Model-1 : Bernoulli Naive Bayes.
y = []
print(len(X))
for x in lines_with_value:
    # print(x[1][0])
    if(x[1][1]):
        index = 0
    else:
        index = 1
    y.append(index)

text_list = []
for article in lines_with_value:
    text_list.append(article[0])
X_train, X_test, y_train, y_test = train_test_split(text_list, y, test_size=0.05, random_state=26105111)
BNBmodel = BernoulliNB()
start = time.time()
BNBmodel.fit(X_train, y_train)
end = time.time()
print("The execution time of this model is {:.2f} seconds\n".format(end-start))
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)


4668


/home/balu/cl2/project/env/lib/python3.8/site-packages/sklearn/utils/validation.py:856: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[list(['టీడీపీకి', '2019లో', 'అగ్రిగోల్డ్\u200c', 'బాధితుల', 'దెబ్బ..', 'టీడీపీకి', '2019', 'ఎన్నికల్లో', 'అగ్రిగోల్డ్\u200c', 'బాధితుల', 'దెబ్బ', 'తగులుతుందని', 'అగ్రిగోల్డ్\u200c', 'బాధితుల', 'సంఘం', 'గౌరవ', 'అధ్యక్షుడు', 'ముప్వాళ్ల', 'నాగేశ్వరరావు', 'అన్నారు.', 'ఆ', 'సందర్భంగా', 'ఆదివారం', 'ఆయన', 'మాట్లాడుతూ..', 'ఈ', 'నెల(మే)', '30', '31', 'తేదీల్లో', 'అగ్రిగోల్డ్\u200c', 'బాధితుల', 'ఆత్మఘోష', 'పాదయాత్ర', 'ఉంటుందని', 'తెలిపారు.', 'అగ్రిగోల్డ్\u200c', 'సమస్యపై', 'వేసిన', 'మంత్రివర్గ', 'ఉపసంఘం', 'నేటికి', 'సమావేశం', 'కాలేదు.', 'ఎన్నో', 'ఆశలతో', 'బాధితులు', 'ఎదురుచూస్తున్నారు.', 'కానీ', 'అగ్రిగోల్డ్\u200c', 'సమస్యలపై', 'పాలకుల్లో', 'ఏ', 'విధమైన', 'చలనం', 'లేదని', 'విమర్శించారు.', 'అందుకు', 'నిరసనగా', 'గుంటూరు', 'నుంచి', 'వెలగపూడి', 'వరకు', '‘ఛలో', 'సెక్రటేరియట్\u200c’', 'కు', 'పిలుపునిచ్చామన్నారు.', 'పోలీసులకు', 'పని', 'లేకుండా', 'శాంతియుతంగా', 'ఈ', 'నిరసన', 'పాదయాత్ర', 'చేపడతాం.', '20', 'లక్షల', 'అగ్రిగోల్డ్\u200c', 'బాధితులపై